# 02 · Candidates Lab — сбор и настройка пула кандидатов 🎯

Цель: быстро получить высокий **Recall@K** на валидации и стабильный пул для ранкера.

Что здесь:
- включение/настройка источников (covis, seq-ngrams, pop/recency, item2vec/ALS, контент, граф);
- быстрый бейзлайн и PANIC;
- Recall@K / NDCG@K, срезы, «дырявые» запросы;
- квоты/веса/дедуп;
- сохранение `candidates.parquet` + `candidates_report.json`.

Все действия time-safe, без утечек по времени.

In [ ]:
from IPython.display import display, HTML
display(HTML("""
<style>
/* ipywidgets v8 (JupyterLab 4) */
.jp-OutputArea .widget-button .widget-label { 
  white-space: normal !important; 
  overflow: visible !important; 
  text-overflow: clip !important;
  line-height: 1.2 !important;
}
/* fallback для ipywidgets v7 */
.jupyter-widgets.widget-button .widget-label {
  white-space: normal !important; 
  overflow: visible !important; 
  text-overflow: clip !important;
  line-height: 1.2 !important;
}
</style>
"""))


In [ ]:
import os, sys, json, time, math, glob, shlex, subprocess, warnings, pathlib
from pathlib import Path
from datetime import datetime
from collections import defaultdict, Counter

import numpy as np
import pandas as pd

from IPython.display import display, clear_output, HTML
import ipywidgets as W

warnings.filterwarnings("ignore")
pd.set_option("display.max_colwidth", 140)

REPO = Path.cwd()
sys.path.insert(0, str(REPO))

def run_cmd(cmd_list, check=True, env=None):
    print("▶", " ".join(shlex.quote(x) for x in cmd_list))
    p = subprocess.Popen(cmd_list, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, env=env)
    try:
        for line in iter(p.stdout.readline, ''):
            print(line, end='')
        p.stdout.close()
        ret = p.wait()
    except KeyboardInterrupt:
        p.terminate(); ret = p.wait(); print("
[aborted]")
    if check and ret != 0:
        raise RuntimeError(f"Command failed ({ret}): {' '.join(cmd_list)}")
    return ret

def run_tool(script_rel, **kwargs):
    args = []
    for k,v in kwargs.items():
        if v is None:
            continue
        if isinstance(v, bool):
            v = "1" if v else "0"
        args += [f"--{k}", str(v)]
    return run_cmd(["python", script_rel] + args)

def now_tag():
    return datetime.now().strftime("%m%d_%H%M")

def profile_name(path):
    return Path(path).stem

def art_paths(dataset_id, profile_path):
    prof = profile_name(profile_path)
    root = Path("artifacts")
    return dict(
        dataio_dir = root / "recsys" / "dataio" / dataset_id,
        profile_dir = root / "recsys" / "dataio" / dataset_id / prof,
        cand_dir = root / "recsys" / "candidates" / dataset_id / prof,
    )

def exists(p):
    try: return Path(p).exists()
    except: return False

In [ ]:
BTN_LAYOUT = W.Layout(min_width="220px", width="auto", height="36px", flex="0 0 auto")
ROW_LAYOUT = W.Layout(flex_flow="row wrap", grid_gap="8px")
GRID_LAYOUT = W.Layout(grid_template_columns="repeat(3, minmax(220px, 1fr))", grid_gap="8px")

### Ячейка 3 — Панель параметров (ipywidgets)

In [ ]:
STATE = dict(
    DATASET_ID = "s5e11",
    PROFILE_PATH = "configs/recsys/profiles/gate.yaml",
    SEED = 42,
    JOBS = -1,
    K_TARGET = 300,
    FAST = True,
    PANIC = False,
)

# Генераторы (включение/отключение)
GEN = dict(
    covis=True,
    session_ngrams=True,
    pop=True,
    recency=True,
    item2vec=False,
    als=False,
    content_tfidf=False,
    graph_ppr=False,
    brand_fallback=True,
    category_fallback=True,
)

# Параметры генераторов
PARAMS = dict(
    windows_days = "3,7,30",   # covis/seq окна (строка: "3,7,30")
    decay_alpha = 0.9,         # экспоненциальный декей (0…1)
    last_k = 5,                # seq-ngrams: учитывать последние K
)

# Виджеты
w_dataset = W.Text(STATE["DATASET_ID"], description="dataset_id:", layout=W.Layout(width="280px"))
w_profile = W.Text(STATE["PROFILE_PATH"], description="profile:", layout=W.Layout(width="520px"))
w_seed    = W.IntText(STATE["SEED"], description="seed:", layout=W.Layout(width="140px"))
w_jobs    = W.IntText(STATE["JOBS"], description="jobs:", layout=W.Layout(width="140px"))
w_k       = W.IntText(STATE["K_TARGET"], description="K target:", layout=W.Layout(width="180px"))
w_fast    = W.Checkbox(STATE["FAST"], description="FAST")
w_panic   = W.Checkbox(STATE["PANIC"], description="PANIC")

toggles = [
    W.Checkbox(GEN["covis"], description="covis"),
    W.Checkbox(GEN["session_ngrams"], description="session_ngrams"),
    W.Checkbox(GEN["pop"], description="pop"),
    W.Checkbox(GEN["recency"], description="recency"),
    W.Checkbox(GEN["item2vec"], description="item2vec"),
    W.Checkbox(GEN["als"], description="ALS/BPR"),
    W.Checkbox(GEN["content_tfidf"], description="content"),
    W.Checkbox(GEN["graph_ppr"], description="graph_PPR"),
    W.Checkbox(GEN["brand_fallback"], description="brand_fallback"),
    W.Checkbox(GEN["category_fallback"], description="category_fallback"),
]

w_windows = W.Text(PARAMS["windows_days"], description="windows_days:", layout=W.Layout(width="260px"))
w_decay   = W.FloatText(PARAMS["decay_alpha"], description="decay_alpha:", layout=W.Layout(width="240px"))
w_lastk   = W.IntText(PARAMS["last_k"], description="last_k:", layout=W.Layout(width="200px"))

btn_apply = W.Button(description="Применить параметры", button_style="success", layout=BTN_LAYOUT)
out_apply = W.Output()

def on_apply(_):
    with out_apply:
        clear_output()
        STATE["DATASET_ID"]  = w_dataset.value.strip()
        STATE["PROFILE_PATH"] = w_profile.value.strip()
        STATE["SEED"]        = int(w_seed.value)
        STATE["JOBS"]        = int(w_jobs.value)
        STATE["K_TARGET"]    = int(w_k.value)
        STATE["FAST"]        = bool(w_fast.value)
        STATE["PANIC"]       = bool(w_panic.value)
        # toggles
        keys = ["covis","session_ngrams","pop","recency","item2vec","als","content_tfidf","graph_ppr","brand_fallback","category_fallback"]
        for k, w in zip(keys, toggles):
            GEN[k] = bool(w.value)
        PARAMS["windows_days"] = w_windows.value.strip()
        PARAMS["decay_alpha"]  = float(w_decay.value)
        PARAMS["last_k"]       = int(w_lastk.value)
        print("STATE:")
        print(json.dumps(STATE, ensure_ascii=False, indent=2))
        print("GEN:")
        print(json.dumps(GEN, ensure_ascii=False, indent=2))
        print("PARAMS:")
        print(json.dumps(PARAMS, ensure_ascii=False, indent=2))

btn_apply.on_click(on_apply)

W.VBox([
    W.HTML("<h3>Параметры и источники</h3>"),
    W.HBox([w_dataset, w_seed, w_jobs, w_k, w_fast, w_panic], layout=ROW_LAYOUT),
    w_profile,
    W.HTML("<b>Источники кандидатов</b>"),
    W.HBox(toggles[:5], layout=ROW_LAYOUT),
    W.HBox(toggles[5:], layout=ROW_LAYOUT),
    W.HTML("<b>Параметры генераторов</b>"),
    W.HBox([w_windows, w_decay, w_lastk], layout=ROW_LAYOUT),
    btn_apply,
    out_apply
])

### Ячейка 4 — Проверка входных данных и размеров

In [ ]:
btn_check = W.Button(description="Проверить входные данные", button_style="primary", layout=BTN_LAYOUT)
out_check = W.Output()

def on_check(_):
    with out_check:
        clear_output()
        A = art_paths(STATE["DATASET_ID"], STATE["PROFILE_PATH"])
        need = [
            A["dataio_dir"]/"interactions_norm.parquet",
            A["dataio_dir"]/"items_norm.parquet",
            A["dataio_dir"]/"splits.json",
            A["profile_dir"]/"pairs_val.parquet",      # вал пары (label=1/0)
            A["profile_dir"]/"queries_val.parquet",    # валидационные запросы
        ]
        ok = True
        for p in need:
            print(("OK  " if exists(p) else "MISS"), p)
            ok &= exists(p)
        if not ok:
            print("
⚠️ Часть артефактов отсутствует. Заверши 01_data_sanity или run_splits.")
            return
        # размеры
        inter = pd.read_parquet(need[0])
        qv = pd.read_parquet(need[4])
        print("
interactions:", inter.shape, "| unique users:", inter.user_id.nunique(), "| items:", inter.item_id.nunique())
        print("queries_val:", qv.shape[0])

btn_check.on_click(on_check)
W.VBox([btn_check, out_check])

### Ячейка 5 — Построение overrides и вызов run_candidates.py

In [ ]:
btn_build = W.Button(description="Собрать кандидатов (по текущим настройкам)", button_style="success", layout=BTN_LAYOUT)
out_build = W.Output()

def build_overrides_json(path_json:str):
    # Формируем «плоский» overrides для tools/run_candidates.py (если он поддерживает --overrides)
    # Если не поддерживает — такие же параметры должны быть в профиле/конфигах.
    srcs = []
    if GEN["covis"]:
        srcs.append(dict(name="covis", windows_days=PARAMS["windows_days"], decay_alpha=PARAMS["decay_alpha"]))
    if GEN["session_ngrams"]:
        srcs.append(dict(name="session_ngrams", last_k=PARAMS["last_k"], windows_days=PARAMS["windows_days"]))
    if GEN["pop"]:
        srcs.append(dict(name="pop"))
    if GEN["recency"]:
        srcs.append(dict(name="recency"))
    if GEN["item2vec"]:
        srcs.append(dict(name="item2vec", train=STATE["FAST"]==False))
    if GEN["als"]:
        srcs.append(dict(name="als", train=STATE["FAST"]==False))
    if GEN["content_tfidf"]:
        srcs.append(dict(name="content_tfidf"))
    if GEN["graph_ppr"]:
        srcs.append(dict(name="graph_ppr"))
    if GEN["brand_fallback"]:
        srcs.append(dict(name="brand_fallback"))
    if GEN["category_fallback"]:
        srcs.append(dict(name="category_fallback"))

    overrides = dict(
        seed=STATE["SEED"],
        jobs=STATE["JOBS"],
        K=STATE["K_TARGET"],
        fast=STATE["FAST"],
        panic=STATE["PANIC"],
        sources=srcs
    )
    Path(path_json).parent.mkdir(parents=True, exist_ok=True)
    Path(path_json).write_text(json.dumps(overrides, ensure_ascii=False, indent=2))
    return overrides

def on_build(_):
    with out_build:
        clear_output()
        A = art_paths(STATE["DATASET_ID"], STATE["PROFILE_PATH"])
        overrides_path = f"artifacts/tmp/{STATE['DATASET_ID']}_{profile_name(STATE['PROFILE_PATH'])}_cands_overrides.json"
        ov = build_overrides_json(overrides_path)
        print("Overrides →", overrides_path)
        t0 = time.time()
        # Вызов генератора кандидатов
        run_tool("recsys/tools/run_candidates.py",
                 dataset_id=STATE["DATASET_ID"],
                 profile=STATE["PROFILE_PATH"],
                 overrides=overrides_path,      # если не поддерживается — убери этот флаг
                 K=STATE["K_TARGET"],
                 jobs=STATE["JOBS"],
                 seed=STATE["SEED"],
                 cache=1)
        dt = time.time() - t0
        print(f"
✓ Готово за {dt:.1f}s")
        # Показать где файл
        cand_path = A["cand_dir"]/"candidates.parquet"
        print("candidates:", cand_path, "exists:", exists(cand_path))

btn_build.on_click(on_build)
W.VBox([btn_build, out_build])

### Ячейка 6 — Подсчёт Recall@K / NDCG@K по валидации

In [ ]:
btn_eval = W.Button(description="Посчитать метрики (Recall/NDCG) на val", button_style="primary", layout=BTN_LAYOUT)
out_eval = W.Output()

def _load_val_and_cands():
    A = art_paths(STATE["DATASET_ID"], STATE["PROFILE_PATH"])
    pairs_val = pd.read_parquet(A["profile_dir"]/"pairs_val.parquet")
    cands = pd.read_parquet(A["cand_dir"]/"candidates.parquet")
    # Ожидаемые поля: query_id, item_id, score, source (source — опционально)
    need = {"query_id","item_id"}
    miss = need - set(cands.columns)
    if miss:
        raise AssertionError(f"candidates.parquet не содержит {miss}")
    return pairs_val, cands

def _recall_ndcg_at_k(pairs_val, cands, ks=(10,20,50,100)):
    # ожидаем по pairs_val: query_id,item_id,label (binary)
    pv = pairs_val[["query_id","item_id","label"]].copy()
    pv["label"] = pv["label"].astype(int)
    pos = pv[pv["label"]==1].groupby("query_id")["item_id"].apply(set).to_dict()
    # сгруппируем кандидатов
    cands = cands.sort_values(["query_id","score"], ascending=[True, False]).copy() if "score" in cands.columns else cands.copy()
    group = cands.groupby("query_id")["item_id"].apply(list)
    results = {}
    for k in ks:
        hits = 0; tot = 0; dcg = 0.0; idcg = 0.0; covered = 0
        for q, true_items in pos.items():
            pred = group.get(q, [])[:k]
            if len(pred)>0: covered += 1
            # Recall@K (или HitRate если 1 позитив)
            hit = int(any(it in true_items for it in pred))
            hits += hit; tot += 1
            # NDCG@K (binary relevance)
            # gain = 1/log2(rank+1) для первого совпадения
            _dcg = 0.0
            for r, it in enumerate(pred, start=1):
                if it in true_items:
                    _dcg = 1.0 / math.log2(r+1); break
            dcg += _dcg
            idcg += 1.0  # т.к. максимум один релевантный (next-item)
        results[f"recall@{k}"] = hits / max(1, tot)
        results[f"ndcg@{k}"] = (dcg / max(1.0, idcg))
        results[f"coverage@{k}"] = covered / max(1, len(pos))
    results["queries"] = len(pos)
    return results

def on_eval(_):
    with out_eval:
        clear_output()
        pv, cd = _load_val_and_cands()
        res = _recall_ndcg_at_k(pv, cd, ks=(10,20,50,100))
        print(json.dumps(res, indent=2))

btn_eval.on_click(on_eval)
W.VBox([btn_eval, out_eval])

### Ячейка 7 — Аналитика источников: вклад, пересечения, «дыры»

In [ ]:
btn_sources = W.Button(description="Разложить вклад источников", button_style="", layout=BTN_LAYOUT)
out_sources = W.Output()

def on_sources(_):
    with out_sources:
        clear_output()
        pv, cd = _load_val_and_cands()
        if "source" not in cd.columns:
            print("В candidates.parquet нет колонки source — вклад по источникам не посчитать.")
            return
        # вклад по попаданиям (хоть одно попадание среди top-K → атрибутнуть «первому источнику, который дал попадание»)
        K = 20
        cd_sorted = cd.sort_values(["query_id","score"], ascending=[True,False])
        grp = cd_sorted.groupby("query_id")
        pos = pv[pv["label"]==1].groupby("query_id")["item_id"].apply(set).to_dict()
        counts = Counter()
        misses = []
        for q, dfq in grp:
            true_items = pos.get(q, set())
            if not true_items:
                continue
            top = dfq.head(K)
            hit_rows = top[top["item_id"].isin(true_items)]
            if len(hit_rows):
                src = hit_rows.iloc[0]["source"] if "source" in hit_rows.columns else "?"
                counts[str(src)] += 1
            else:
                misses.append(q)
        total = len(pos)
        rows = [{"source": s, "hit_queries": c, "share": c/max(1,total)} for s,c in counts.most_common()]
        df = pd.DataFrame(rows)
        display(df if len(df) else "нет попаданий")
        print(f"Queries with no hit in top@{K}: {len(misses)}/{total}")
        if len(misses):
            miss_df = pd.DataFrame({"query_id": misses[:50]})
            display(miss_df)

btn_sources.on_click(on_sources)
W.VBox([btn_sources, out_sources])

### Ячейка 8 — Настройка covis / seq-ngrams и перезапуск частично

In [ ]:
btn_tune_covis = W.Button(description="Пересчитать covis/seq с новыми окнами", button_style="warning", layout=BTN_LAYOUT)
out_tune = W.Output()

def on_tune_covis(_):
    with out_tune:
        clear_output()
        A = art_paths(STATE["DATASET_ID"], STATE["PROFILE_PATH"])
        overrides_path = f"artifacts/tmp/{STATE['DATASET_ID']}_{profile_name(STATE['PROFILE_PATH'])}_cands_overrides.json"
        ov = build_overrides_json(overrides_path)
        print("New windows:", PARAMS["windows_days"], "last_k:", PARAMS["last_k"])
        run_tool("recsys/tools/run_candidates.py",
                 dataset_id=STATE["DATASET_ID"],
                 profile=STATE["PROFILE_PATH"],
                 overrides=overrides_path,
                 K=STATE["K_TARGET"],
                 jobs=STATE["JOBS"],
                 seed=STATE["SEED"],
                 only="covis,session_ngrams",   # если поддерживается: пересчитать частично
                 cache=1)
        print("✓ Пересчёт covis/seq завершён")
        # метрики
        pv, cd = _load_val_and_cands()
        res = _recall_ndcg_at_k(pv, cd, ks=(10,20,50,100))
        print(json.dumps(res, indent=2))

btn_tune_covis.on_click(on_tune_covis)
W.VBox([btn_tune_covis, out_tune])

### Ячейка 9 — Включение item2vec/ALS/контент/граф и обновление пула

In [ ]:
btn_heavy = W.Button(description="Добавить item2vec/ALS/контент/граф (если включены)", button_style="warning", layout=BTN_LAYOUT)
out_heavy = W.Output()

def on_heavy(_):
    with out_heavy:
        clear_output()
        overrides_path = f"artifacts/tmp/{STATE['DATASET_ID']}_{profile_name(STATE['PROFILE_PATH'])}_cands_overrides.json"
        ov = build_overrides_json(overrides_path)
        print("heavy sources flags:", {k:v for k,v in GEN.items() if k in ["item2vec","als","content_tfidf","graph_ppr"]})
        run_tool("recsys/tools/run_candidates.py",
                 dataset_id=STATE["DATASET_ID"],
                 profile=STATE["PROFILE_PATH"],
                 overrides=overrides_path,
                 K=STATE["K_TARGET"],
                 jobs=STATE["JOBS"],
                 seed=STATE["SEED"],
                 only="item2vec,als,content_tfidf,graph_ppr",  # если поддерживается
                 cache=1)
        print("✓ Обновлён пул heavy-источниками")
        pv, cd = _load_val_and_cands()
        res = _recall_ndcg_at_k(pv, cd, ks=(10,20,50,100))
        print(json.dumps(res, indent=2))

btn_heavy.on_click(on_heavy)
W.VBox([btn_heavy, out_heavy])

### Ячейка 10 — Квоты по источникам и локальный реранж пула

In [ ]:
btn_quotas = W.Button(description="Применить квоты по источникам и пересчитать метрики", button_style="info", layout=BTN_LAYOUT)
out_quotas = W.Output()

# простой UI для квот (задаём после загрузки кандидатов)
w_quota_text = W.Text("covis:140,session_ngrams:80,pop:40,recency:40,item2vec:60,als:60,content_tfidf:40,graph_ppr:40,brand_fallback:30,category_fallback:30",
                      description="quotas:", layout=W.Layout(width="900px"))

def _apply_quotas(cands: pd.DataFrame, quotas: dict, K_target: int):
    if "source" not in cands.columns:
        # без источников — просто обрежем до K
        return cands.sort_values(["query_id","score"], ascending=[True,False]).groupby("query_id").head(K_target).reset_index(drop=True)
    out_rows = []
    for q, dfq in cands.groupby("query_id", sort=False):
        parts = []
        for s, g in dfq.sort_values("score", ascending=False).groupby("source"):
            k = int(quotas.get(str(s), 0))
            if k>0:
                parts.append(g.head(k))
        if parts:
            merged = pd.concat(parts, axis=0)
        else:
            merged = dfq.sort_values("score", ascending=False)
        merged = merged.drop_duplicates(subset=["item_id"], keep="first").head(K_target)
        merged["query_id"] = q
        out_rows.append(merged)
    res = pd.concat(out_rows, axis=0).reset_index(drop=True)
    return res

def on_quotas(_):
    with out_quotas:
        clear_output()
        pv, cd = _load_val_and_cands()
        if "source" not in cd.columns:
            print("Нет source в candidates — квоты недоступны.")
            return
        # распарсим строку вида "name:k,name:k,..."
        quotas = {}
        for tok in w_quota_text.value.split(","):
            tok = tok.strip()
            if not tok: continue
            if ":" in tok:
                name, val = tok.split(":")
                quotas[name.strip()] = int(val.strip())
        new_cd = _apply_quotas(cd, quotas, STATE["K_TARGET"])
        print("До:", cd.groupby("query_id").size().describe()[["mean","min","max"]].to_dict())
        print("После:", new_cd.groupby("query_id").size().describe()[["mean","min","max"]].to_dict())
        # локальная оценка
        res = _recall_ndcg_at_k(pv, new_cd, ks=(10,20,50,100))
        print(json.dumps(res, indent=2))
        # предложим сохранить как «локальную» версию
        A = art_paths(STATE["DATASET_ID"], STATE["PROFILE_PATH"])
        out_p = A["cand_dir"] / "candidates_local_quota.parquet"
        new_cd.to_parquet(out_p, index=False)
        print("✓ candidates_local_quota.parquet сохранён →", out_p)

btn_quotas.on_click(on_quotas)
W.VBox([w_quota_text, btn_quotas, out_quotas])

### Ячейка 11 — PANIC-режим (минимум источников, максимум стабильности)

In [ ]:
btn_panic = W.Button(description="⚡ PANIC: covis+pop+fallback (K=300)", button_style="danger", layout=BTN_LAYOUT)
out_panic = W.Output()

def on_panic(_):
    with out_panic:
        clear_output()
        # жёсткие параметры для паника
        GEN["covis"]=True; GEN["session_ngrams"]=True; GEN["pop"]=True; GEN["recency"]=True
        GEN["item2vec"]=GEN["als"]=GEN["content_tfidf"]=GEN["graph_ppr"]=False
        GEN["brand_fallback"]=GEN["category_fallback"]=True
        STATE["K_TARGET"]=max(200, STATE["K_TARGET"])
        PARAMS["windows_days"]="7,30"; PARAMS["last_k"]=5; PARAMS["decay_alpha"]=0.9
        overrides_path = f"artifacts/tmp/{STATE['DATASET_ID']}_{profile_name(STATE['PROFILE_PATH'])}_panic_overrides.json"
        build_overrides_json(overrides_path)
        run_tool("recsys/tools/run_candidates.py",
                 dataset_id=STATE["DATASET_ID"],
                 profile=STATE["PROFILE_PATH"],
                 overrides=overrides_path,
                 K=STATE["K_TARGET"],
                 jobs=STATE["JOBS"],
                 seed=STATE["SEED"],
                 cache=1)
        pv, cd = _load_val_and_cands()
        res = _recall_ndcg_at_k(pv, cd, ks=(10,20,50,100))
        print("PANIC metrics:")
        print(json.dumps(res, indent=2))

btn_panic.on_click(on_panic)
W.VBox([btn_panic, out_panic])

### Ячейка 12 — Паспорт кандидатов и сохранение отчёта

In [ ]:
btn_pass = W.Button(description="Сохранить candidates_report.json", button_style="success", layout=BTN_LAYOUT)
out_pass = W.Output()

def on_pass(_):
    with out_pass:
        clear_output()
        A = art_paths(STATE["DATASET_ID"], STATE["PROFILE_PATH"])
        pv, cd = _load_val_and_cands()
        metrics = _recall_ndcg_at_k(pv, cd, ks=(10,20,50,100))
        report = dict(
            dataset_id=STATE["DATASET_ID"],
            profile=STATE["PROFILE_PATH"],
            seed=STATE["SEED"],
            jobs=STATE["JOBS"],
            k_target=STATE["K_TARGET"],
            fast=STATE["FAST"],
            panic=STATE["PANIC"],
            generators=GEN,
            params=PARAMS,
            metrics=metrics,
            saved_at=datetime.utcnow().isoformat()+"Z"
        )
        out_json = A["cand_dir"] / "candidates_report.json"
        A["cand_dir"].mkdir(parents=True, exist_ok=True)
        out_json.write_text(json.dumps(report, ensure_ascii=False, indent=2))
        print("✓ report saved:", out_json)
        print(json.dumps(metrics, indent=2))

btn_pass.on_click(on_pass)
W.VBox([btn_pass, out_pass])

### Ячейка 13 — Просмотр артефактов кандидатов

In [ ]:
btn_list = W.Button(description="Показать файлы кандидатов", button_style="", layout=BTN_LAYOUT)
out_list = W.Output()

def on_list(_):
    with out_list:
        clear_output()
        A = art_paths(STATE["DATASET_ID"], STATE["PROFILE_PATH"])
        d = A["cand_dir"]
        if not d.exists():
            print("Папка ещё не создана:", d)
            return
        files = sorted(d.rglob("*"))
        df = pd.DataFrame([{"path":str(p), "size_MB": round(p.stat().st_size/1024/1024,3)} for p in files if p.is_file()])
        display(df)

btn_list.on_click(on_list)
W.VBox([btn_list, out_list])

### Ячейка 14 — Быстрый one-click сценарий

In [ ]:
btn_all = W.Button(description="🚀 Быстрый прогон: check → build → eval → sources → save report", button_style="success", layout=BTN_LAYOUT)
out_all = W.Output()

def on_all(_):
    with out_all:
        clear_output()
        try:
            # 1) check
            on_check(None)
            # 2) build
            on_build(None)
            # 3) eval
            on_eval(None)
            # 4) sources
            on_sources(None)
            # 5) save report
            on_pass(None)
        except Exception as e:
            print("ОШИБКА:", e)
            raise

btn_all.on_click(on_all)
W.VBox([btn_all, out_all])

### Пороговые ориентиры
- **Recall@20** (next-item): целимся ≥ 0.90; PANIC — ≥ 0.88.
- Queries без попаданий ≤ 2–3%. Если выше — включи `brand/category fallback`.
- Если Recall высок, смело **снижай K_target** (ускоришь фичи/ранкер).
- Сильные просадки на CS-срезах → добавляй `content`/`graph`.

**Дальше:** `03_ranker_lab.ipynb` — фичи + ранкер + анализ OOF, затем `04_blend_eval` для реранка и мета-бленда.